<center>

# Trabajo Práctico Final
## Procesamiento de Imágenes Biomédicas
### Primer Cuatrimestre 2024


### _Diseño de un algoritmo automatizado para la detección de epilepsia en tiempo real mediante EEG_





</center>

**Docentes:**

- Roberto Sebastián Tomás
- Aylin Agatha Vazquez Chenlo
- Francisco Tassara
- Victoria Reppucci

**Alumnos:**

- Bajlec, Ivo - 62175
- Grau, Gonzalo Andrés - 62259
- Neira, Lucas Matias - 61570



# ESTADO DEL ARTE

# PROBLEMÁTICA

# OBJETIVO Y DISEÑO

# DESARROLLO

## Técnicas de acondicionmiento de señales biomédicas

# CONCLUSIÓN


ESTADO DEL ARTE:
PROBLEMÁTICA: 
OBJETIVOS y DISEÑO:
DESARROLLO: 
1. Técnicas de acondicionamiento de señales biomédicas / Técnicas de mejoramiento de
imágenes biomédicas
2. Algoritmos de procesamiento de señales/imágenes biomédicas vistas en la materia para
extraer información relevante de la señal/imagen. Se pueden implementar técnicas vistas
en clase pero no con el fin habitual visto en clase.
3. Técnica novedosa de procesamiento. Por ejemplo Transformada Wavelets, Independent
Component Analysis, Principal Component Analysis, etc (ya sea como acondicionamien-
to como para extraer características para la creación del modelo de AA o RN)
4. Creación de 2 modelos de clasificación/regresión distintos que utilicen distintos tipos de
técnicas de AA o RN para resolver la problemática planteada.
RESULTADOS:
CONCLUSIÓN: 